In [1]:
import numpy as np
from scipy.fftpack import dct
import matplotlib.pyplot as plt
import scipy.io.wavfile as wav
import math

In [2]:
def mfcc(file):
    sample_rate, signal = wav.read(file)
    signal = signal[0:int(5 * sample_rate)] # Keep 1st 5 secs of signal
    numCoefficients = 13 # choose the size of mfcc array
    frame_size = .025 #25ms frames
    frame_separation = .015 #10ms overlap
    
    frame_len_dist = int(round(frame_size * sample_rate))
    frame_sep_dist = int(round(frame_separation * sample_rate))
    
    framed_matrix = frame(signal, sample_rate, frame_len_dist, frame_sep_dist)
    
    framed_matrix = smooth(framed_matrix, frame_len_dist)
    
    power_matrix = getPeriodogram(framed_matrix, frame_len_dist)
    #print(frame_len_dist)
    #print(power_matrix.shape)
    
    
    minHz = 0
    maxHz = sample_rate / 2
    mlfb = getMelFilterBank(minHz, maxHz, frame_len_dist, sample_rate)
    #print(mlfb.shape)
    
    coefficients = getCoefficients(power_matrix, mlfb)
    return coefficients

In [3]:
#FRAMING
def frame(signal, sample_rate, frame_len_dist, frame_sep_dist):
    frame_overlap_dist = frame_len_dist - frame_sep_dist
    signal_length = len(signal)
    num_frames = int(np.floor(signal_length / frame_sep_dist))
    if num_frames * frame_sep_dist + frame_overlap_dist < signal_length:
        num_frames += 1
    corr_signal_length = (num_frames - 1) * frame_sep_dist + frame_len_dist
    zero_padding_length = corr_signal_length - signal_length
    zero_padding = np.zeros(zero_padding_length)
    padded_signal = np.append(signal, zero_padding)
    framed_matrix = np.zeros((num_frames, frame_len_dist))
    for frame_num in range(num_frames):
        for i in range(int(frame_len_dist)):
            framed_matrix[frame_num][i] = padded_signal[frame_num * frame_sep_dist + i]
    return framed_matrix

In [4]:
#WINDOWING y(n) = x(n) * w(n)
def smooth(framed_matrix, frame_len_dist):
    framed_matrix *= np.hamming(frame_len_dist)
    return framed_matrix

In [5]:
#FFT Matrix i.e. periodogram
def getPeriodogram(framed_matrix, frame_len_dist):
    framed_matrix = np.absolute(np.fft.fft(framed_matrix))
    framed_matrix = (1/frame_len_dist) * np.square(framed_matrix)
    return framed_matrix


In [6]:
#Mel to frequency conversions
def freqToMel(freq):
    return 1127.01048 * np.log(1 + freq / 700.0)

def melToFreq(mel):
    return 700 * (np.exp(mel / 1127.01048 ) - 1)

In [7]:
#Generate Filter Bank
#Algorithm to generate filter bank from 
def getMelFilterBank(minHz, maxHz, frame_len_dist, sample_rate, numFilters = 40):
    minMel = freqToMel(minHz)
    maxMel = freqToMel(maxHz)
    
    melAxis = np.linspace(minMel, maxMel, numFilters + 2)
    hzAxis = melToFreq(melAxis)
    roundedHzAxis = np.floor((frame_len_dist+1)*hzAxis/sample_rate)
    #print(roundedHzA)
    
    melfb = np.zeros((numFilters, frame_len_dist))

    for m in range(1, numFilters + 1):
        #Iterate through every row of the filter bank to populate it: note most of them will be 0s
        #m - row; k - column
        left = int(roundedHzAxis[m - 1])   # left
        middle = int(roundedHzAxis[m])             # center
        right = int(roundedHzAxis[m + 1])    # right

        for k in range(left, middle):
            melfb[m - 1, k] = (k - roundedHzAxis[m - 1]) / (roundedHzAxis[m] - roundedHzAxis[m - 1])
        for k in range(middle, right):
            melfb[m - 1, k] = (roundedHzAxis[m + 1] - k) / (roundedHzAxis[m + 1] - roundedHzAxis[m])
    return melfb


In [8]:
#Extract Coefficients after Taking DCT
def getCoefficients(power_matrix, melfb, num_coefficients = 12 ):
    filter_banks = np.dot(power_matrix, melfb.T)
    #print(filter_banks.shape)
    filter_banks = 20 * np.log10(filter_banks)  # dB
    mfcc = dct(filter_banks, type=2, axis=1, norm='ortho')[:, 1 : (num_coefficients + 1)] # Keep 2-13
    return mfcc

In [9]:
file = "train/s1.wav"
coeffs = mfcc(file)
print(coeffs.shape)
print(coeffs)

(69, 12)
[[ -6.85303117e+00   7.49932619e+00   3.61713872e+00  -1.18732663e+00
   -7.33016329e+00   3.51671415e+00  -3.80246665e+00   3.61781083e+00
   -3.80107866e+00   7.29182884e-01   1.05613968e+01   1.33430329e+01]
 [  5.32898969e+00   1.14970794e+01   1.30723900e+00  -3.20347659e+00
   -3.98180926e+00   5.90541073e+00  -7.00002805e+00  -7.42650813e+00
   -3.82496894e+00   3.04582708e+00   1.51539661e+01   1.91138865e+01]
 [  2.22770205e+00   1.70798043e+01   1.08009954e+01   3.21760479e+00
   -2.60689266e+00   9.57389908e+00   1.69114814e+00  -2.17922795e+00
   -3.53389144e+00  -1.08952040e+01  -1.48939998e+00   8.95039773e+00]
 [ -2.82649501e+00   1.82740379e+01  -7.03263576e+00  -7.25543935e+00
   -2.73470589e+00   1.13380070e+01  -5.63923727e+00   1.86032394e+00
   -6.36380803e-02   2.41663011e+00   1.34936339e+01   1.59962265e+01]
 [ -3.81189796e+00   1.44611586e+01   2.01747878e+00   3.23042028e+00
   -7.37788018e-01   1.01664824e+01   1.72103530e+01   3.65275758e-01
    1.1

In [10]:
from scipy.cluster.vq import vq, kmeans


In [11]:
def train(folder, k = 16, iter = 100, numfiles = 8):
    codebooks = {}
    for i in range(1,numfiles + 1):
        file = str(folder) + "/s" + str(i) + ".wav"
        obs = mfcc(file)
        codebook = kmeans(obs, k, iter, thresh=1e-8)
        codebooks[i] = codebook[0]
    return codebooks


In [12]:
def test(file, codebooks, numfiles = 8):
    min_dist = float('inf')
    best_match = 0
    for i in range(1, numfiles + 1):
        obs = mfcc(file)
        codebook = codebooks[i]
        code, dist = vq(obs, codebook)
        avg_dist = np.average(dist)
        print(avg_dist)
        if avg_dist < min_dist:
            min_dist = avg_dist
            best_match = i
    return best_match
        
    

In [13]:
codebooks = train("train")
#print(codebooks[1])
test("test/s5.wav", codebooks)

67.379988398
55.9562230445
70.5462878255
72.1250218091
43.2609098817
65.9020082131
61.38429358
65.7050034095


5

In [14]:
#Speakers
#1 - Afrikaans Woman 1: http://accent.gmu.edu/searchsaa.php?function=detail&speakerid=1
#2 - Arabic Woman 2: http://accent.gmu.edu/searchsaa.php?function=detail&speakerid=23
#3 - Dutch Man: http://accent.gmu.edu/searchsaa.php?function=detail&speakerid=1300
#4 - Hindi Man: http://accent.gmu.edu/searchsaa.php?function=detail&speakerid=910
#5 - Japanese Woman: http://accent.gmu.edu/searchsaa.php?function=detail&speakerid=223
#6 - Spanish Man: http://accent.gmu.edu/searchsaa.php?function=detail&speakerid=323

In [16]:
codebooks = train("accent-data/train", numfiles = 6)
#print(codebooks[1])
test("accent-data/test/s6.wav", codebooks, numfiles=6)

97.7004076749
114.619033844
102.730983776
81.3785519743
88.0047141285
40.5350232475


6

In [116]:
#print(numFilters)
#print(melfb.T.shape)
#print(framed_matrix.shape)
#print(mfcc.shape)
#print(np.avg([1,3,4]))

In [ ]:
#FINISHED WITH SIGNAL PROCESSING